imports

In [1]:
import tensorflow as tf
#import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


2024-09-30 14:43:18.273334: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Helper and preprocessing functions

In [2]:
def tf_scale_imgs(imgs, scale_factor):
    return imgs * scale_factor + 0.5 * (1 - scale_factor) * tf.ones(imgs.shape)


def tf_scale_labels(labels, scale_factor):
    return labels * scale_factor + 0.5 * (1 - scale_factor) * tf.ones(labels.shape)


def tf_f_inv(x, act_fn):
    """ (activation_size, batch_size) """
    if act_fn == "LINEAR":
        m = x
    elif act_fn == "TANH":
        num = tf.ones_like(x) + x
        div = tf.ones_like(x) - x + 1e-7
        m = 0.5 * tf.math.log(num / div)
    elif act_fn == "LOGSIG":
        div = tf.ones_like(x) - x + 1e-7
        m = tf.math.log((x / div) + 1e-7)
    else:
        raise ValueError(f"{act_fn} not supported")
    return m


def boring_movie(x, y, n_steps=4):
  """Stacks a Tensor to create a 'time series' of repeating images."""
  x = tf.stack([x for _ in range(n_steps)], axis=1)  # make the "boring" movie of subsequently following images
  y = tf.stack([y for _ in range(n_steps)], axis=1)
  return x, y


def random_omissions(x, y, omissions=0.5):
  """Randomly omits a fraction of the images in a batch.""" # why needed? -> not needed here 
  mask = tf.random.uniform(tf.shape(x)[:2], 0, 1)[:, :, None, None, None] > omissions
  x = x * tf.cast(mask, dtype=tf.float32)
  return x, y


def img_preproc(x, y, dtype=tf.float32): # add: tf.image.resize(image, [28,28])
  """Cast input image to a certain tf dtype and normalize them between 0 and 1."""
  x = tf.cast(x, dtype) / 255.
  #x = tf_scale_imgs(x, cf.img_scale)
  #y = tf_scale_labels(y, cf.label_scale)
  #x = tf_f_inv(x, "TANH")
  #y = tf.one_hot(y, depth=10)
  return x, y


def flatten(x, y):
  """flattens a video image series (or batch of images) to (n_batch, n_steps, 1) d."""
  shape = tf.shape(x)
  if len(shape) == 5: # hack, determining if it's a video or not
    x = tf.reshape(x, [shape[0], shape[1], -1])
  elif len(shape) == 4:
    x = tf.reshape(x, [shape[0], -1])
  return x, y




n_steps = 4

build the network

In [3]:

class CustomDense(tf.keras.layers.Dense):
    def call(self, inputs):
        """This works like a dense, except for the activation being called earlier."""
        # Apply the activation to the input first
        activated_input = self.activation(inputs)
        # Perform the matrix multiplication and add the bias
        output = tf.matmul(activated_input, self.kernel)
        if self.use_bias:
            output = output + self.bias
        return output


class PredictiveCodingNetwork(tf.keras.Sequential):
    def __init__(self, layers, vars, beta, **kwargs):
        """Initialize a PredictiveCodingNetwork"""
        super().__init__(layers, **kwargs)
        self.vars = tf.convert_to_tensor(vars, dtype=tf.float32)
        self.beta = beta

    def call_with_states(self, x):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        x_list = [x]
        for layer in self.layers:
            x = layer(x)
            x_list.append(x)
        return x_list

    def train_step(self, data):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        # do the stuff we do in train_epochs
        outputs, errors = self.infer(x, y)
        self.update_params(outputs, errors)

        # Update metrics (includes the metric that tracks the loss)
        pred = self.call(x[:, 0])
        for metric in self.metrics:
            metric.update_state(y[:, 0], pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def infer(self, x_batch, y_batch):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        errors = [None for _ in range(len(self.layers))]
        f_x_arr = [None for _ in range(len(self.layers))]
        f_x_deriv_arr = [None for _ in range(len(self.layers))]
        shape = x_batch.shape
        batch_size, n_steps = shape[0], shape[1]

        for itr in range(n_steps):
            # always update the current forward call
            x_forward_cur = self.call_with_states(x_batch[:, itr])
            x_forward_cur[-1] = y_batch[:, itr]
            # if its the first itr, set x to the current forward call
            if itr == 0:
                x = x_forward_cur
            if itr != 0:
                # update g and x only for consecutive iterations
                for l in range(1, len(self.layers)):
                    g = tf.multiply(tf.matmul(errors[l], self.layers[l].kernel, transpose_b=True), f_x_deriv_arr[l])
                    x[l] = x[l] + (x_forward_cur[l] - x_forward_prev[l]) + self.beta * (-errors[l-1] + g)
            # update f_x etc for every iteration
            for l in range(len(self.layers)):
                f_x = self.layers[l].activation(x[l])
                f_x_deriv_fn = self.get_activation_derivative(self.layers[l].activation)
                f_x_deriv = f_x_deriv_fn(x[l])
                f_x_arr[l] = f_x
                f_x_deriv_arr[l] = f_x_deriv
                errors[l] = (x[l + 1] - tf.matmul(f_x, self.layers[l].kernel) - self.layers[l].bias) / self.vars[l]
            # fill the old forward pass with the current forward pass
            x_forward_prev = x_forward_cur
        return x, errors

    def update_params(self, x, errors):
        """Update the model parameters."""
        batch_size = tf.cast(tf.shape(x[0])[0], tf.float32)
        gradients = []
        for l, layer in enumerate(self.layers):
            grad_w = self.vars[-1] * (1 / batch_size) * tf.matmul(tf.transpose(self.layers[l].activation(x[l])), errors[l])
            grad_b = self.vars[-1] * (1 / batch_size) * tf.reduce_sum(errors[l], axis=0)
            gradients.append((-grad_w, layer.kernel))
            gradients.append((-grad_b, layer.bias))
        self.optimizer.apply_gradients(gradients)

    def get_activation_derivative(self, activation):
        """Return a function for the derivative of the given activation function."""
        activation_fn = tf.keras.activations.get(activation)
        if activation_fn == tf.keras.activations.linear:
            return lambda x: tf.ones_like(x)
        elif activation_fn == tf.keras.activations.tanh:
            return lambda x: 1 - tf.square(tf.nn.tanh(x))
        elif activation_fn == tf.keras.activations.sigmoid:
            return lambda x: tf.nn.sigmoid(x) * (1 - tf.nn.sigmoid(x))
        else:
            raise ValueError(f"{activation} not supported")


Load the data

In [4]:
# load the dataset
train_dataset = tfds.load("mnist", split='train', as_supervised=True)
val_dataset = tfds.load("mnist", split='test', as_supervised=True)

# define batch_size
batch_size = 1024

# give it one pass to test
show_idx = 333
for x, y in train_dataset.batch(batch_size).map(img_preproc).map(boring_movie).map(lambda x,y:random_omissions(x, y, omissions=0.5)).map(flatten).take(1):

    # plot one sequence
    fig, axes = plt.subplots(1, x.shape[1], figsize=(15, 4.1), sharey=True)
    for idx in range(x.shape[1]):
      axes[idx].imshow(tf.reshape(x[show_idx][idx], [28, 28, 1]), cmap="gray")
      axes[idx].get_xaxis().set_visible(False)
      axes[idx].get_yaxis().set_visible(False)
      axes[idx].set_title(f"Step {idx}")

    # set title
    plt.suptitle(f"Showing input video sequence of sample {show_idx}")
    plt.show()

NameError: name 'tfds' is not defined

In [1]:
# Build dataset

# 9 categories: Church, barn  beach  castle  cave  conference_room  forest  library  restaurant
# leading: Barn, beach, library, restaurant, cave 
# trailing: Church, conference room, library, forest

img_dir = '/scratch/guetlid95/datasets/mcdermott_2024/Stimuli_2/'
img_dir_lead = '/scratch/guetlid95/datasets/mcdermott_2024/Stimuli_2/Leading/'
img_dir_trail = '/scratch/guetlid95/datasets/mcdermott_2024/Stimuli_2/Trailing/' 
class_names_L = ['barn', 'beach', 'cave', 'library', 'restaurant']
class_names_T = ['Church', 'castle', 'conference_room', 'forest']
batch_size = 32
image_size = (28,28)
validation_split = 0.1
seed = 123

data_leading = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_lead, 
    label_mode = 'int',
    class_names= class_names_L,
    color_mode = 'rgb',
    image_size = image_size, 
    shuffle = True, 
    #seed = seed
)

data_trailing = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_trail, 
    label_mode = 'int',
    class_names= class_names_T,
    color_mode = 'rgb', 
    image_size = image_size, 
    shuffle = True, 
    #seed = seed
)


NameError: name 'tf' is not defined

In [4]:
# Create a leading and trailing dataset with tensorflow

img_dir_lead = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Leading/'
img_dir_trail = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Trailing/'
class_names_L = ['barn', 'beach', 'cave', 'library', 'restaurant']
class_names_T = ['Church', 'castle', 'conference_room', 'forest']
batch_size = None # adjust if needed, e.g., 32
image_size = (28,28)
validation_split = 0.1
seed = 123

data_leading = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_lead, 
    label_mode = 'int',
    class_names= class_names_L,
    batch_size = batch_size,
    color_mode = 'rgb',
    image_size = image_size, 
    shuffle = True, 
    seed = seed
)

data_trailing = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_trail, 
    label_mode = 'int',
    class_names= class_names_T,
    batch_size = batch_size,
    color_mode = 'rgb', 
    image_size = image_size, 
    shuffle = True, 
    seed = seed
)

# Check class names and corresponding labels
print("Leading imgs categories:", data_leading.class_names)
print("Trailing imgs categories:", data_trailing.class_names)
   
class_names_dL = data_leading.class_names
for index, class_name in enumerate(class_names_dL):
    print(f"Category: '{class_name}'; Label: {index}")
        
class_names_dT = data_trailing.class_names
for index, class_name in enumerate(class_names_dT ):
    print(f"Category: '{class_name}'; Label: {index}")

Found 1796 files belonging to 5 classes.
Found 1796 files belonging to 4 classes.
Leading imgs categories: ['barn', 'beach', 'cave', 'library', 'restaurant']
Trailing imgs categories: ['Church', 'castle', 'conference_room', 'forest']
Category: 'barn'; Label: 0
Category: 'beach'; Label: 1
Category: 'cave'; Label: 2
Category: 'library'; Label: 3
Category: 'restaurant'; Label: 4
Category: 'Church'; Label: 0
Category: 'castle'; Label: 1
Category: 'conference_room'; Label: 2
Category: 'forest'; Label: 3


In [5]:
# Create a dict that assigns the correct labels for each leading-trailing img pair

"""
L1 = barn = label 0 
L2 = beach = label 1
L3 = library = label 3
L4 = restaurant = label 4 
L5 = cave = label 2

T6 = Church = label 0 
T7 = conference room = label 2
T8 = castle = label 1
T9 = forest = label 3

MAPPING:

L1 -> T6 = 0.75 -> (0,0)
L1 -> T7 = 0.25 -> (0,2)
L1 -> T8 = 0 -> (0,1)
L1 -> T9 = 0 -> (0,3)

L2 -> T6 = 0.75 -> (1,0)
L2 -> T7 = 0.25 -> (1,2)
L2 -> T8 = 0 -> (1,1)
L2 -> T9 = 0 -> (1,3)

L3 -> T6 = 0 -> (3,0)
L3 -> T7 = 0 -> (3,2)
L3 -> T8 = 0.5 -> (3,1)
L3 -> T9 = 0.5 -> (3,3)

L4 -> T6 = 0.25 -> (4,0)
L4 -> T7 = 0.75 -> (4,2)
L4 -> T8 = 0 -> (4,1)
L4 -> T9 = 0 -> (4,3)

L5 -> T6 = 0.25 -> (2,0)
L5 -> T7 = 0.75 -> (2,2)
L5 -> T8 = 0 -> (2,1)
L5 -> T9 = 0 -> (2,3)

"""

label_dict = {
    (0, 0): 0.0,
    (0, 2): 0.0,
    (0, 1): 0.25,
    (0, 3): 0.75,
    (1, 0): 0.0,
    (1, 2): 0.0,
    (1, 1): 0.25,
    (1, 3): 0.75,
    (3, 0): 0.0,
    (3, 2): 0.0,
    (3, 1): 0.25,
    (3, 3): 0.75,
    (4, 0): 0.0,
    (4, 2): 0.0,
    (4, 1): 0.25,
    (4, 3): 0.75,
    (2, 0): 0.0,
    (2, 2): 0.0,
    (2, 1): 0.5,
    (2, 3): 0.5
}


def imgsequence(img_t1, img_t2, label_t1, label_t2, label_dict, batch_size=None): #data_t1, data_t2, label_dict, batch_size=None
    """
    Function that stacks images and prepares labels for each batch.
    Now takes direct batch inputs instead of iterators.
    """
    
    # go through each img and label in the leading & trailing batch 
    #img_t1, label_t1 = next(iter(data_t1)) # next(iter(data_t1.batch(batch_size)))
    #img_t2, label_t2 = next(iter(data_t2))# next(iter(data_t2.batch(batch_size)))
    
    img_t1 = tf.cast(img_t1, dtype=tf.float32)
    img_t2 = tf.cast(img_t2, dtype=tf.float32)
    #print(type(img_t1))
    
    #print("Shape of img_t1:", img_t1.shape)
    #print("Shape of img_t2:", img_t2.shape)
    #print("Shape of label_t1:", label_t1.shape)
    #print("Shape of label_t2:", label_t2.shape)
    
    
    # stack imgs from both datatsets into sequence of img
    x = tf.stack([img_t1, img_t1, img_t2, img_t2], axis = 1)
    #print("Type x:", x.dtype)
    #print("Shape of x after stacking:", x.shape)
    
    y = [label_dict.get((int(img_t1), int(img_t2)), 0) for img_t1, img_t2 in zip(label_t1, label_t2)]    
    #print("Current labels:", label_t1, label_t2)
    #print("Type y before:", y.dtype)
    
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    
    if y.shape[0] != batch_size:
        raise ValueError(f"Y tensor size mismatch: expected {batch_size}, got {y.shape[0]}")
    
    y = tf.reshape(y, (batch_size, 1))
    #print("Type y:", type(y))
    #print("Shape y :", y.shape)
    #print("Type of x:", type(x), "Shape of x:", x.shape, "dtype of x:", x.dtype)
    #print("Type of y:", type(y), "Shape of y:", y.shape, "dtype of y:", y.dtype)
    
    #print(y)
    #print(x)
    
    return x,y 


In [2]:
def generate_dataset(data_t1, data_t2, label_dict, batch_size=None):
    
    # iterate through leading and trailing datasets and batch them accordingly 
    leading = iter(data_t1.batch(batch_size))
    trailing = iter(data_t2.batch(batch_size))

    while True:
        try:
            
            img_t1, label_t1 = next(leading)
            img_t2, label_t2 = next(trailing)

            # Check if both current batches == batch_size, if not stop generating the dataset 
            if img_t1.shape[0] == batch_size and img_t2.shape[0] == batch_size:
                
                x, y = imgsequence(img_t1, img_t2, label_t1, label_t2, label_dict, batch_size)
                #print("test x:", x.shape)
                #print("test y:", y.shape)  
                yield x, y
            else:
                print(f"Skipping batch: leading batch size = {img_t1.shape[0]}, trailing batch size = {img_t2.shape[0]}")
                break
        
        except StopIteration:
            # Break when there are no more samples
            break
        

# Create new dataset 
dataset = tf.data.Dataset.from_generator(
    lambda: generate_dataset(data_leading, data_trailing, label_dict, batch_size=32),
    output_signature=(
        tf.TensorSpec(shape=(32, 4, 28, 28, 3), dtype=tf.float32),  # shape of x
        tf.TensorSpec(shape=(32, 1), dtype=tf.float32)  # shape of y 
    )
)

print(dataset)

for x, y in dataset:
    print("Shape of x:", x.shape)
    print("Shape of y:", y.shape)
    print("Type of x:", x.dtype)
    print("Type of y:", y.dtype)
    

NameError: name 'tf' is not defined

In [6]:
def split_data(dataset, validation_split=0.1, shuffle=True, seed=123):
    """Splits a dataset into training and validation datasets."""
    dataset_size = dataset.cardinality().numpy()  # Get the dataset size
    val_size = int(validation_split * dataset_size)
    train_size = dataset_size - val_size
    
    # Shuffle the dataset
    if shuffle:
        dataset = dataset.shuffle(buffer_size=dataset_size, seed=seed)
    
    # Split the dataset
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size)
    
    return train_dataset, val_dataset

In [8]:
batch_size = 32
n_steps = 4
train_leading, val_leading = split_data(data_leading, validation_split=0.1)
train_trailing, val_trailing = split_data(data_trailing, validation_split=0.1)

train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_dataset(train_leading, train_trailing, label_dict, batch_size=batch_size),
    output_signature=(
        tf.TensorSpec(shape=(batch_size, n_steps, 28, 28, 3), dtype=tf.float32),  # shape of x
        tf.TensorSpec(shape=(batch_size, 1), dtype=tf.float32)  # shape of y 
    )
)

val_dataset_leading = val_leading.batch(batch_size)
val_dataset_trailing = val_trailing.batch(batch_size)

# Combine leading and trailing datasets for validation
val_dataset = tf.data.experimental.sample_from_datasets([val_dataset_leading, val_dataset_trailing])

print("Trainset:", train_dataset)
print("Valset:", val_dataset)

Trainset: <_FlatMapDataset element_spec=(TensorSpec(shape=(32, 4, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.float32, name=None))>
Valset: <_DirectedInterleaveDataset element_spec=(TensorSpec(shape=(None, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [14]:
dataset_size = sum(1 for _ in dataset)
train_size = int(0.9 * dataset_size)
val_size = dataset_size - train_size
print(f"Validation dataset size: {val_size}")

# Split into train and validation datasets
train_dataset = dataset.take(train_size)  # Take the first 90% for training
val_dataset = dataset.skip(train_size).take(val_size)

#print(train_dataset.element_spec)
#print(val_dataset.element_spec)


batch_count = 0
for _ in train_dataset:
    batch_count += 1
print(f"Total number of batches in train dataset: {batch_count}")

batch_count = 0
for _ in val_dataset:
    batch_count += 1
print(f"Total number of batches in val dataset: {batch_count}")

Skipping batch: leading batch size = 4, trailing batch size = 4
Validation dataset size: 6
(TensorSpec(shape=(32, 4, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.float32, name=None))
(TensorSpec(shape=(32, 4, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.float32, name=None))


2024-09-27 16:14:57.752873: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-27 16:14:59.514863: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Total number of batches in train dataset: 50
Total number of batches in val dataset: 6


2024-09-27 16:15:01.450268: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


train the network

In [6]:
# define the model
model = PredictiveCodingNetwork([CustomDense(units=500, activation="tanh"),
                                 CustomDense(units=500, activation="tanh"),
                                 CustomDense(units=10, activation="tanh")],
                                vars=[1, 1, 1], # variances. This is super useless and in the code only the last variance is used
                                beta=0.1)

model.build([None, 784])
model.compile(optimizer=tf.keras.optimizers.AdamW(),
              metrics=["accuracy"],
              loss="CategoricalCrossentropy",  # This is just a sham loss we need so model.evaluate doesn't throw an error. We don't use it.
              )

# evaluate accuracy and train model
loss, accuracy = model.evaluate(val_dataset.map(img_preproc).map(flatten))
print(f"Accuracy before training: {accuracy}")

model.fit(train_dataset.shuffle(99999).map(img_preproc).map(flatten),
          #validation_data=val_dataset.batch(batch_size).map(img_preproc).map(flatten),
          epochs=3)

loss, accuracy = model.evaluate(val_dataset.map(img_preproc).map(flatten))
print(f"Accuracy after training: {accuracy}")

# tensorboard -> history 

I0000 00:00:1724673876.088574 3559536 service.cc:146] XLA service 0x14a478004190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724673876.088603 3559536 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-08-26 14:04:36.406478: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-26 14:04:37.005512: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 1/10 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.1396 - loss: 7.0209

I0000 00:00:1724673877.877965 3559536 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.1404 - loss: 7.1597
Accuracy before training: 0.1421000063419342
Epoch 1/3
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.6300 - loss: 0.5808
Epoch 2/3
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7755 - loss: 0.4985
Epoch 3/3
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7872 - loss: 0.5350
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8369 - loss: 0.8267 
Accuracy after training: 0.838699996471405


In [72]:

tt = test_dataset.shuffle(99999).map(img_preproc).map(flatten)

for x_batch, y_batch in tt.take(1):
    input_shape = x_batch.shape
    #print("Input Shape:", input_shape)
    print("Batch Images (x_batch):", tf.reduce_min(x_batch), tf.reduce_max(x_batch))
    print("Batch Images (y_batch):", tf.reduce_min(y_batch), tf.reduce_max(y_batch))

    

Skipping batch: leading batch size = 4, trailing batch size = 4
Batch Images (x_batch): tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
Batch Images (y_batch): tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(0.75, shape=(), dtype=float32)


2024-09-27 10:31:50.397762: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
print(test_dataset.element_spec)

batch_count = 0
for _ in test_dataset:
    batch_count += 1
print(f"Total number of batches: {batch_count}")

(TensorSpec(shape=(32, 4, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 1), dtype=tf.float32, name=None))
Skipping batch: leading batch size = 4, trailing batch size = 4
Total number of batches: 56


2024-09-27 16:11:34.272440: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
